# Asyncio High-Level Review
- Writing concurrent code using the async/await syntax

- Coroutines and Tasks
    - Thing we strive to make async/await
    - Needs an entry-point: asyncio.run() -> top-level main

- Awaitable: things with an await dunder
    - Three types: coroutines, Tasks, Futures

- Coroutine definition
    1. A function: async def signature
    2. Object: returned by calling a coroutine function

- Future: low-level awaitable object which represents an eventual result of an async op

## Methods of Interest
    - Creating Tasks
        1. asyncio.create_task(coro, *, name=None, context=None)
            - If a reference isn't saved, a fire and forget could lose the task
        2. asyncio.TaskGroup -> context manager way
            - Downside: waits for all tasks in the group to finish
            - *Downside/Plus: early exits don't raise the asyncio.CancelledError exception, tons of exceptions to this rule!!
                - Due to this, managing through create_task is better/more granular!
    - Running Tasks
        1. asyncio.run(coro, *, debug=None, loop_factory=None)
        2. class asyncio.Runner(*, debug=None, loop_factory=None)
            - Ways to operate
                - run(coro, *, context=None)
                - close()
                - get_loop()
            - Notes
                - Runner uses the lazy initialization strategy, its constructor doesn’t initialize underlying low-level structures
                - Embedded loop and context are created at the with body entering or the first call of run() or get_loop()
                - Installs a custom signal.SIGINT handler before any user code is executed and removes it when exiting from the function
        3. [Within an asyncio function] awaitable asyncio.gather(*aws, return_exceptions=False)
            - Same as creating tasks number 2: dont do it. This way you don't have to use shielding

    - Waiting
        1. The asyncio.timeout() context manager is what transforms the asyncio.CancelledError into a TimeoutError 
        2. async asyncio.wait_for(aw, timeout)
        3. asyncio.as_completed(aws, *, timeout=None) -> returns an iterable
            - Async iterator supplied
            - Avoid if possible as the task completion makes it difficult 

    - Introspection
        1. asyncio.current_task(loop=None) -> returns the currently running Task instance, or None
        2. asyncio.iscoroutine(obj) -> returns bool

## High-Level Task Object
- class asyncio.Task(coro, *, loop=None, name=None, context=None, eager_start=False)
- A Future-like object that runs a Python coroutine. Not threadsafe!
- Used to run coroutines in event loops
    - If a coroutine awaits on a Future, the Task suspends the execution of the coroutine and waits for the completion of the Future
    - When the Future is done, the execution of the wrapped coroutine resumes
- asyncio.Task inherits from Future all of its APIs except Future.set_result() and Future.set_exception()
- Some methods: done(), result(), get_name(), set_name(value), cancel(msg=None)

## Event Loop
- Event loops use cooperative scheduling (on Task at a time)
- Use the high-level asyncio.create_task() function to create Tasks, or the low-level loop.create_task() or ensure_future() functions.
- add_done_callback(callback, *, context=None)

## Future Addition
- Call Graph Introspection (3.14)
- asyncio.to_thread(...)
- asyncio.run_coroutine_threadsafe(...)
- Low-Level Futures and Event Loop (good for tail tracking of a file)


In [8]:
import asyncio
import time

from asyncio import TaskGroup

In [ ]:
ipv4_connect = asyncio.create_task(asyncio.open_connection("127.0.0.1", 80))
ipv6_connect = asyncio.create_task(asyncio.open_connection("::1", 80))
tasks = [ipv4_connect, ipv6_connect]

async for earliest_connect in asyncio.as_completed(tasks):
    # earliest_connect is done. The result can be obtained by
    # awaiting it or calling earliest_connect.result()
    reader, writer = await earliest_connect

    if earliest_connect is ipv6_connect:
        print("IPv6 connection established.")
    else:
        print("IPv4 connection established.")

In [ ]:
async def main():
    await asyncio.sleep(1)
    print('hello')

with asyncio.Runner() as runner:
    runner.run(main())

In [ ]:
async def main():
    try:
        async with asyncio.timeout(10):
            await long_running_task()
    except TimeoutError:
        print("The long operation timed out, but we've handled it.")

    try:
        await asyncio.wait_for(eternity(), timeout=1.6)
    except TimeoutError:
        print('timeout!')

    print("This statement will run regardless.")

In [ ]:
async def display_date():
    loop = asyncio.get_running_loop()
    end_time = loop.time() + 5.0
    while True:
        print(datetime.datetime.now())
        if (loop.time() + 1.0) >= end_time:
            break
        await asyncio.sleep(1)

In [9]:
async def job(task_id, sleep_time):
    print(f'Task {task_id}: start')
    await asyncio.sleep(sleep_time)
    print(f'Task {task_id}: done')

In [10]:
async def main():
    async with asyncio.TaskGroup() as tg:
        task1 = tg.create_task(
            job(1, 1))

        task2 = tg.create_task(
            job(2, 2))

        print(f"started at {time.strftime('%X')}")

    # The await is implicit when the context manager exits.

    print(f"finished at {time.strftime('%X')}")

In [11]:
class TerminateTaskGroup(Exception):
    """Exception raised to terminate a task group."""

async def force_terminate_task_group():
    """Used to force termination of a task group."""
    raise TerminateTaskGroup()

async def main():
    try:
        async with TaskGroup() as group:
            # spawn some tasks
            group.create_task(job(1, 0.5))
            group.create_task(job(2, 1.5))
            # sleep for 1 second
            await asyncio.sleep(1)
            # add an exception-raising task to force the group to terminate
            group.create_task(force_terminate_task_group())
    except* TerminateTaskGroup:
        pass

# Usually asyncio.run(main()) required but not in a notebook
main()

<coroutine object main at 0x75507c593d80>